Use IMDB dataset packaged with Keras. It has already been labelled, where each word has been turned into an integer.

In [6]:
import numpy as np
from keras.datasets import imdb

# Set allow_pickle to True since new version defaults to False
old = np.load
np.load = lambda *a,**k: old(*a, allow_pickle=True, **k)

# Keep only 10k most commonly used words, rest discarded
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

print ('Training Data: ', len(train_data), 'Testing Data: ', len(test_data))
print ('Training Data Example: ', train_data[3], ' Label: ', test_labels[3])

np.load = old
del(old)

TypeError: ignored